<a id='top'></a>

# Criar arquivos com as cotações trimestrais

### Imports

In [64]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import time
from unidecode import unidecode
from pandas_datareader import data
from pandas_datareader import data as web

import warnings
#warnings.simplefilter(action='ignore', category=FutureWarning)

### Obter cotações trimestrais

In [56]:
years = [2019]
years = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

#Carregar todos os arquivos em um único dataframe
df_aux = pd.DataFrame({'A' : []})
df_full = pd.DataFrame({'A' : []})

for year in years:
    
    print(year)
    
    path = '../data_processed/'+str(year)+'/bpa_'+str(year)+'.csv'
    df_bpa = pd.read_csv(path, sep=',', encoding = "ISO-8859-1")
    
    path = '../data_processed/'+str(year)+'/bpp_'+str(year)+'.csv'
    df_bpp = pd.read_csv(path, sep=',', encoding = "ISO-8859-1")
    
    path = '../data_processed/'+str(year)+'/dre_'+str(year)+'.csv'
    df_dre = pd.read_csv(path, sep=',', encoding = "ISO-8859-1")
    
    path = '../data_processed/'+str(year)+'/dfc_mi_'+str(year)+'.csv'
    df_dfc_mi = pd.read_csv(path, sep=',', encoding = "ISO-8859-1")
    
    path = '../data_processed/'+str(year)+'/dva_'+str(year)+'.csv'
    df_dva = pd.read_csv(path, sep=',', encoding = "ISO-8859-1")
    
    if df_full.empty:
        df_full = df_bpa.copy()
        df_full = pd.concat([df_full, df_bpp], axis=1, ignore_index=False)
        df_full = pd.concat([df_full, df_dre], axis=1, ignore_index=False)
        df_full = pd.concat([df_full, df_dfc_mi], axis=1, ignore_index=False)
        df_full = pd.concat([df_full, df_dva], axis=1, ignore_index=False)

        #Remover colunas duplicadas
        df_full = df_full.loc[:,~df_full.columns.duplicated()]
    else:
        df_aux = pd.DataFrame({'A' : []})
        df_aux = df_bpa.copy()
        df_aux = pd.concat([df_aux, df_bpp], axis=1, ignore_index=False)
        df_aux = pd.concat([df_aux, df_dre], axis=1, ignore_index=False)
        df_aux = pd.concat([df_aux, df_dfc_mi], axis=1, ignore_index=False)
        df_aux = pd.concat([df_aux, df_dva], axis=1, ignore_index=False)
        
        #Remover colunas duplicadas
        df_aux = df_aux.loc[:,~df_aux.columns.duplicated()]
        
        df_full = pd.concat([df_full, df_aux], axis=0, ignore_index=True)
        
#Ordenar por "ticker" e "dt_refer"
df_full.sort_values(by=['ticker', 'dt_refer'], inplace=True, ignore_index=True)

2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


In [100]:
def get_stock_data(ticker, start_date, end_date, count):
    """
    Gets historical stock data of given tickers between dates
    :param ticker: company, or companies whose data is to fetched
    :type ticker: string or list of strings
    :param start_date: starting date for stock prices
    :type start_date: string of date "YYYY-mm-dd"
    :param end_date: end date for stock prices
    :type end_date: string of date "YYYY-mm-dd"
    :return: stock_data.csv
    """
    
    all_data = pd.DataFrame() 
    try:
        all_data = web.get_data_yahoo(ticker, start_date, end_date, interval='d')
    except Exception as e:
        
        count += 1        
        mess = str(e).replace("\'","")
        if mess != 'Date':
            print('ERROR: ' + mess + '\n')
            print("No success to fetch the data.\n")
            return
        
        if count < 3:
            print("Yahoo error. Trying after 5 seconds")
            time.sleep(5)
            get_stock_data(ticker, start_date, end_date, count)
        else: 
            if count == 3:
                print("Tried 3 times, Yahoo error. Trying after 60 seconds")
                time.sleep(60)
                get_stock_data(ticker, start_date, end_date, count)
            else:
                print("No success to fetch the data.\n")
                return
    return all_data

In [46]:
#df_full = df_full[ (df_full.ticker == 'BEEF3') & (df_full.dt_refer >= '2011-01-01')]
df_full = df_full[ (df_full.ticker == 'BEEF3')]

In [101]:
#Criar novas colunas para armazenar preços de fechamentos e volumes
df_full['adj_close'] = np.nan
df_full['volume'] = np.nan

#Obter lista de tickers
tickers = df_full.ticker.unique()

i = 0

for ticker in tickers:

    print(ticker)
    
    #Obter datas trimestrais por ticker
    dt_refers = df_full[df_full['ticker'] == ticker]['dt_refer'].tolist()
    
    for dt_refer in dt_refers:
        
        print(dt_refer)
        
        try:
                
            dt_aux = dt.datetime.strptime(dt_refer, '%Y-%m-%d')
            dt_aux = dt_aux.replace(day=1)            
                        
            #Obter cotações para a data (corrente) e ticker
            df_ticker_act = get_stock_data(ticker +'.SA', dt_aux, dt_refer, 0)
            
            if len(df_ticker_act) != 0:
                adj_close, volume = df_ticker_act[['Adj Close', 'Volume']].iloc[df_ticker_act.shape[0]-1]

                #Adicionar Adj Close e Volume para a data (corrente) e ticker
                idx = df_full[ (df_full['ticker'] == ticker) & (df_full['dt_refer'] == dt_refer)].index[0]            
                df_full.loc[idx, 'adj_close'] = round(adj_close, 2)
                df_full.loc[idx, 'volume'] = round(volume, 2)                    
                        
        except Exception as e:
            print(e)
            mess = str(e).replace("\'","")
            print('ERROR: ' + mess + '\n')
            if mess == 'Date':
                continue
            else:
                break
    i = i + 1
    print('###################################\n')

AALR3
2016-03-31
Yahoo error. Trying after 5 seconds
Yahoo error. Trying after 5 seconds
Tried 3 times, Yahoo error. Trying after 60 seconds
No success to fetch the data.

2016-06-30
Yahoo error. Trying after 5 seconds
Yahoo error. Trying after 5 seconds
Tried 3 times, Yahoo error. Trying after 60 seconds
No success to fetch the data.

2016-09-30
Yahoo error. Trying after 5 seconds
Yahoo error. Trying after 5 seconds
Tried 3 times, Yahoo error. Trying after 60 seconds
No success to fetch the data.

2017-03-31
2017-06-30
2017-09-30
2018-03-31
2018-06-30
2018-09-30
2019-03-31
2019-06-30
2019-09-30
2020-03-31
###################################

ABEV3
2013-06-30
2013-09-30
2014-03-31
2014-06-30
2014-09-30
2015-03-31
2015-06-30
2015-09-30
2016-03-31
2016-06-30
2016-09-30
2017-03-31
2017-06-30
2017-09-30
2018-03-31
2018-06-30
2018-09-30
2019-03-31
2019-06-30
2019-09-30
2020-03-31
###################################

ADHM3
2012-03-31
2012-06-30
2012-09-30
2013-03-31
2013-06-30
2013-09-30
201

2018-09-30
2019-03-31
2019-06-30
2019-09-30
2020-03-31
###################################

ATMP3
2011-03-31
Yahoo error. Trying after 5 seconds
Yahoo error. Trying after 5 seconds
Tried 3 times, Yahoo error. Trying after 60 seconds
No success to fetch the data.

2011-06-30
Yahoo error. Trying after 5 seconds
Yahoo error. Trying after 5 seconds
Tried 3 times, Yahoo error. Trying after 60 seconds
No success to fetch the data.

2011-09-30
Yahoo error. Trying after 5 seconds
Yahoo error. Trying after 5 seconds
Tried 3 times, Yahoo error. Trying after 60 seconds
No success to fetch the data.

2012-03-31
Yahoo error. Trying after 5 seconds
Yahoo error. Trying after 5 seconds
Tried 3 times, Yahoo error. Trying after 60 seconds
No success to fetch the data.

2012-06-30
Yahoo error. Trying after 5 seconds
Yahoo error. Trying after 5 seconds
Tried 3 times, Yahoo error. Trying after 60 seconds
No success to fetch the data.

2012-09-30
Yahoo error. Trying after 5 seconds
Yahoo error. Trying after

2016-06-30
2016-09-30
2017-03-31
2017-06-30
2017-09-30
2018-03-31
2018-06-30
2018-09-30
2019-03-31
2019-06-30
2019-09-30
###################################

BEEF3
2011-03-31
2011-06-30
2011-09-30
2012-03-31
2012-06-30
2012-09-30
2013-03-31
2013-06-30
2013-09-30
2014-03-31
2014-06-30
2014-09-30
2015-03-31
2015-06-30
2015-09-30
2016-03-31
2016-06-30
2016-09-30
2017-03-31
2017-06-30
2017-09-30
2018-03-31
2018-06-30
2018-09-30
2019-03-31
2019-06-30
2019-09-30
2020-03-31
###################################

BIOM3
2011-03-31
Yahoo error. Trying after 5 seconds
Yahoo error. Trying after 5 seconds
Tried 3 times, Yahoo error. Trying after 60 seconds
No success to fetch the data.

2011-06-30
2011-09-30
2012-03-31
2012-06-30
2012-09-30
2013-03-31
2013-06-30
2013-09-30
2014-03-31
2014-06-30
2014-09-30
2015-03-31
2015-06-30
2015-09-30
2016-03-31
2016-06-30
2016-09-30
2017-03-31
2017-06-30
2017-09-30
2018-03-31
2018-06-30
2018-09-30
2019-03-31
2019-06-30
2019-09-30
2020-03-31
######################

No success to fetch the data.

2012-03-31
2012-06-30
2012-09-30
2013-03-31
2013-06-30
2013-09-30
2014-03-31
2014-06-30
2014-09-30
2015-03-31
2015-06-30
2015-09-30
2016-03-31
2016-06-30
2016-09-30
2017-03-31
2017-06-30
2017-09-30
2018-03-31
2018-06-30
2018-09-30
2019-03-31
2019-06-30
2019-09-30
2020-03-31
###################################

BRKM3
2011-03-31
2011-06-30
2011-09-30
2012-03-31
2012-06-30
2012-09-30
2013-03-31
2013-06-30
2013-09-30
2014-03-31
2014-06-30
2014-09-30
2015-03-31
2015-06-30
2015-09-30
2016-03-31
2016-06-30
2016-09-30
2017-03-31
2017-06-30
2017-09-30
2018-03-31
2018-06-30
2018-09-30
2019-03-31
2019-06-30
2019-09-30
2020-03-31
###################################

BRML3
2011-03-31
2011-06-30
2011-09-30
2012-03-31
2012-06-30
2012-09-30
2013-03-31
2013-06-30
2013-09-30
2014-03-31
2014-06-30
2014-09-30
2015-03-31
2015-06-30
2015-09-30
2016-03-31
2016-06-30
2016-09-30
2017-03-31
2017-06-30
2017-09-30
2018-03-31
2018-06-30
2018-09-30
2019-03-31
2019-06-30
2019-09-30
202

2011-06-30
2011-09-30
2012-03-31
2012-06-30
2012-09-30
2013-03-31
2013-06-30
2013-09-30
2014-03-31
2014-06-30
2014-09-30
2015-03-31
2015-06-30
2015-09-30
2016-03-31
2016-06-30
2016-09-30
2017-03-31
2017-06-30
2017-09-30
2018-03-31
2018-06-30
2018-09-30
2019-03-31
2019-06-30
2019-09-30
2020-03-31
###################################

CCRO3
2011-03-31
2011-06-30
2011-09-30
2012-03-31
2012-06-30
2012-09-30
2013-03-31
2013-06-30
2013-09-30
2014-03-31
2014-06-30
2014-09-30
2015-03-31
2015-06-30
2015-09-30
2016-03-31
2016-06-30
2016-09-30
2017-03-31
2017-06-30
2017-09-30
2018-03-31
2018-06-30
2018-09-30
2019-03-31
2019-06-30
2019-09-30
2020-03-31
###################################

CEAB3
2019-03-31
Yahoo error. Trying after 5 seconds
Yahoo error. Trying after 5 seconds
Tried 3 times, Yahoo error. Trying after 60 seconds
No success to fetch the data.

2019-06-30
Yahoo error. Trying after 5 seconds
Yahoo error. Trying after 5 seconds
Tried 3 times, Yahoo error. Trying after 60 seconds
No succe

...

2017-12-31
2018-03-31
2018-09-30
2018-12-31
2019-03-31
2019-09-30
2019-12-31
2020-03-31
###################################

QUAL3
2011-03-31
Yahoo error. Trying after 5 seconds
Yahoo error. Trying after 5 seconds
Tried 3 times, Yahoo error. Trying after 60 seconds
No success to fetch the data.

2011-06-30
2011-09-30
2012-03-31
2012-06-30
2012-09-30
2013-03-31
2013-06-30
2013-09-30
2014-03-31
2014-06-30
2014-09-30
2015-03-31
2015-06-30
2015-09-30
2016-03-31
2016-06-30
2016-09-30
2017-03-31
2017-06-30
2017-09-30
2018-03-31
2018-06-30
2018-09-30
2019-03-31
2019-06-30
2019-09-30
2020-03-31
###################################

QUSW3
2015-06-30
ERROR: No data fetched for symbol QUSW3.SA using YahooDailyReader

No success to fetch the data.

object of type 'NoneType' has no len()
ERROR: object of type NoneType has no len()

###################################

RADL3
2012-03-31
2012-06-30
2012-09-30
2013-03-31
2013-06-30
2013-09-30
2016-03-31
2016-06-30
2016-09-30
2017-03-31
2017-06-30
2017-0

2019-06-30
2019-09-30
2020-03-31
###################################

SCAR3
2011-03-31
2011-06-30
2011-09-30
2012-03-31
2012-06-30
2012-09-30
2013-03-31
2013-06-30
2013-09-30
2014-03-31
2014-06-30
2014-09-30
2015-03-31
2015-06-30
2015-09-30
2016-03-31
2016-06-30
2016-09-30
2017-03-31
2017-06-30
2017-09-30
2018-03-31
2018-06-30
2018-09-30
2019-03-31
2019-06-30
2019-09-30
2020-03-31
###################################

SEER3
2013-03-31
Yahoo error. Trying after 5 seconds
Yahoo error. Trying after 5 seconds
Tried 3 times, Yahoo error. Trying after 60 seconds
No success to fetch the data.

2013-06-30
Yahoo error. Trying after 5 seconds
Yahoo error. Trying after 5 seconds
Tried 3 times, Yahoo error. Trying after 60 seconds
No success to fetch the data.

2013-09-30
Yahoo error. Trying after 5 seconds
Yahoo error. Trying after 5 seconds
Tried 3 times, Yahoo error. Trying after 60 seconds
No success to fetch the data.

2014-03-31
2014-06-30
2014-09-30
2015-03-31
2015-06-30
2015-09-30
2016-03-3

2012-06-30
2012-09-30
2013-03-31
2013-06-30
2013-09-30
2014-03-31
2014-06-30
2014-09-30
2015-03-31
2015-06-30
2015-09-30
2016-03-31
2016-06-30
2016-09-30
2017-03-31
2017-06-30
2017-09-30
2018-03-31
2018-06-30
2018-09-30
2019-03-31
2019-06-30
2019-09-30
2020-03-31
###################################

STKF3
2011-03-31
Yahoo error. Trying after 5 seconds
Yahoo error. Trying after 5 seconds
Tried 3 times, Yahoo error. Trying after 60 seconds
No success to fetch the data.

2011-06-30
Yahoo error. Trying after 5 seconds
Yahoo error. Trying after 5 seconds
Tried 3 times, Yahoo error. Trying after 60 seconds
No success to fetch the data.

2011-09-30
Yahoo error. Trying after 5 seconds
Yahoo error. Trying after 5 seconds
Tried 3 times, Yahoo error. Trying after 60 seconds
No success to fetch the data.

2012-03-31
Yahoo error. Trying after 5 seconds
Yahoo error. Trying after 5 seconds
Tried 3 times, Yahoo error. Trying after 60 seconds
No success to fetch the data.

2012-06-30
Yahoo error. Tryin

2017-09-30
2018-03-31
2018-06-30
2018-09-30
2019-03-31
2019-06-30
2019-09-30
2020-03-31
###################################

TAEE3
2011-03-31
Yahoo error. Trying after 5 seconds
Yahoo error. Trying after 5 seconds
Tried 3 times, Yahoo error. Trying after 60 seconds
No success to fetch the data.

2011-06-30
Yahoo error. Trying after 5 seconds
Yahoo error. Trying after 5 seconds
Tried 3 times, Yahoo error. Trying after 60 seconds
No success to fetch the data.

2011-09-30
Yahoo error. Trying after 5 seconds
Yahoo error. Trying after 5 seconds
Tried 3 times, Yahoo error. Trying after 60 seconds
No success to fetch the data.

2012-03-31
2012-06-30
2012-09-30
2013-03-31
2013-06-30
2013-09-30
2014-03-31
2014-06-30
2014-09-30
2015-03-31
2015-06-30
2015-09-30
2016-03-31
2016-06-30
2016-09-30
2017-03-31
2017-06-30
2017-09-30
2018-03-31
2018-06-30
2018-09-30
2019-03-31
2019-06-30
2019-09-30
2020-03-31
###################################

TASA3
2011-03-31
Yahoo error. Trying after 5 seconds
Yahoo 

2018-09-30
2019-03-31
2019-06-30
2019-09-30
2020-03-31
###################################

TXRX3
2011-03-31
Yahoo error. Trying after 5 seconds
Yahoo error. Trying after 5 seconds
Tried 3 times, Yahoo error. Trying after 60 seconds
No success to fetch the data.

2011-06-30
2011-09-30
Yahoo error. Trying after 5 seconds
Yahoo error. Trying after 5 seconds
Tried 3 times, Yahoo error. Trying after 60 seconds
No success to fetch the data.

2012-03-31
2012-06-30
2012-09-30
2013-03-31
2013-06-30
2013-09-30
2014-03-31
2014-06-30
2014-09-30
2015-03-31
2015-06-30
2015-09-30
2016-03-31
2016-06-30
2016-09-30
2017-03-31
2017-06-30
2017-09-30
2018-03-31
2018-06-30
2018-09-30
2019-03-31
2019-06-30
2019-09-30
2020-03-31
###################################

UCAS3
2013-03-31
2013-06-30
2013-09-30
2014-03-31
2014-06-30
2014-09-30
2015-03-31
2015-06-30
2015-09-30
2016-03-31
2016-06-30
2016-09-30
2017-03-31
2017-06-30
2017-09-30
2018-03-31
2018-06-30
2018-09-30
2019-03-31
2019-06-30
2019-09-30
2020-03-31

In [102]:
df_full.shape

(6524, 433)

In [104]:
#Deletar linhas onde a coluna 'adj_close' eh NaN
df_full = df_full[~df_full['adj_close'].isnull()]

In [105]:
df_full.shape

(5419, 433)

In [106]:
#Salvar dataframe em um arquivo .csv
df_full.to_csv('../data_trimester_close_price/bpa_bpp_dre_dfc_mi_dva_close_price_'+ str(years[0]) + '-' 
               + str(years[len(years)-1]) +'.csv', encoding='utf-8', index=False)